# GEMSS on your custom dataset

This notebook demonstrates how to apply GEMSS to your own dataset with unknown ground truth. It largely mirrors the workflow of demo.ipynb, where applicable.

Use this notebook to try out GEMSS on your custom data. Follow the instructions below to set up the algorithm's parameters.

When an experiment is run, the hyperparameters, search history and results are saved into a predefined directory (variable "experiment_id") and can be loaded again by this notebook (variable "run_or_load").

In [ ]:
from IPython.display import display, Markdown
import os
import numpy as np
from plotly import io as pio

import gemss.config as C
from gemss.config import display_current_config
from gemss.utils.utils import (
    show_solution_summary,
    display_feature_lists,
    save_feature_lists_txt,
    save_feature_lists_json,
    save_selector_history_json,
    save_constants_json,
    load_selector_history_json,
    load_constants_json,
)
from gemss.feature_selection.inference import BayesianFeatureSelector
from gemss.utils.visualizations import (
    show_label_histogram,
    show_final_alphas,
    show_features_in_components,
    show_algorithm_progress,
)
from gemss.postprocessing.result_postprocessing import (
    get_features_from_solutions,
    get_unique_features,
    recover_solutions,
    show_solution_summary,
    show_unique_features_from_full_solutions,
)
from gemss.postprocessing.simple_regressions import (
    solve_any_regression,
    show_regression_metrics,
)
from gemss.diagnostics.recommendations import display_parameter_adjustment_summary
from gemss.utils.utils import show_solution_summary

from gemss.data_handling.data_processing import (
    load_data,
    get_df_from_X,
    preprocess_features,
)

pio.renderers.default = "notebook_connected"  # Ensures plotly plots show in notebooks

# Your setup

In this section, define the specifics for your data and parameters for the feature selection algorithm.

## Dataset parameters

Set up the name of your dataset and the names of the index and label columns.

In [ ]:
# the CSV file should be in the ../data/ directory
# the index and label column names must be included in the dataset
csv_dataset_name = "my_custom_dataset.csv"
index_column_name = "sample ID"
label_column_name = "label"

## Data processing parameters

- **Handling missing values:** By default, only rows with missing labels/response values are dropped and other missing values are handled natively. Other options are available.
- **Handling non-numerical features:** Currently, only numerical features are supported.
- **Feature scaling:** If None, major changes in parameter settings might be necessary. It is highly advised to normalize the data.

In [ ]:
# NA handling options
# Options are:
# - "response": drop rows with NA in the response column only (default).
# - "all": drop rows with NA in any column.
# - "none": do not drop any rows.
dropna_columns = "response"

# the threshold for allowed percentage of missing values in each feature
# features with a higher percentage of missing values will be dropped
# either None or a fraction or a value between 0 and 100
allowed_missing_percentage = 20

# If True, keep only numerical features
drop_non_numeric_features = True

# Optionally, apply scaling to features
# Used in both feature selection algorithm
# and the simple regression models on the results
apply_scaling = "standard"  # options: "standard", "minmax", None

### Choose whether to run the optimization or load saved results

Either run the feature selector (the history and configuration are saved) or load a specified experiment.

This part enables a user to run, save and view multiple experiments on their custom dataset.

In [ ]:
# 'run' => run the feature selector + save the history and configuration
# 'load' => load the optimization configuration + history from file
run_or_load = "run"

# select experiment number
experiment_id = 1

# recommended place to to save your experiments (retrieved by reruns or evaluation of results)
experiment_dir = f"./results/experiment_{experiment_id}"
os.makedirs(experiment_dir, exist_ok=True)
history_filename = f"{experiment_dir}/search_history_results.json"
constants_filename = f"{experiment_dir}/search_setup.json"
features_filename_json = f"{experiment_dir}/all_candidate_solutions.json"

### Govern verbosity and outputting

In [ ]:
# Show plots of algorithm progress over iterations
show_search_history = True

# Choose overall verbosity for various outputs
verbose = True

# Whether to run regressions for every set of solutions and show the summary metrics
run_regressions_for_solutions = True

# one chosen type of candidate solutions to be analyzed in greater detail (for each component)
# can be one of these:
# None   ... no detailed analysis
# 'Full features' ... solutions consisting of all features above the provided minimal |mu| threshold
# 'Top features' ... the top few features according to |mu| values. Number determined by 'desired sparsity'
# 'Outlier features (STD_{deviation})'
#       ... {deviation} must be a number in the OUTLIER_DEVIATION_THRESHOLDS list set up below
#       ... e.g. 'Outlier features (STD_3.0)' for solutions given by outlier detection with 3.0*STD threshold
chosen_solution_type = None

## Load your data

In [ ]:
# Do not change this cell

if verbose:
    display(Markdown("#### Loading your data"))

df, response = load_data(
    csv_dataset_name,
    index_column_name,
    label_column_name,
)

X, y, feature_to_name = preprocess_features(
    df,
    response,
    dropna=dropna_columns,
    allowed_missing_percentage=allowed_missing_percentage,
    drop_non_numeric_features=drop_non_numeric_features,
    apply_scaling=apply_scaling,
    verbose=verbose,
)
df = get_df_from_X(X, feature_to_name)

overall_nan_ratio = np.isnan(X).sum() / (X.shape[0] * X.shape[1])
if verbose:
    display(Markdown("#### Your data:"))
    display(Markdown(f"- **Number of labels:** {len(np.unique(y))}"))
    show_label_histogram(y)
    display(Markdown(f"- **Number of samples:** {X.shape[0]}"))
    display(Markdown(f"- **Number of features:** {X.shape[1]}"))
    display(Markdown(f"{list(feature_to_name.values())}"))
    display(
        Markdown(
            f"- **Total percentage of missing values:** {overall_nan_ratio * 100:.1f} %"
        )
    )

## Set parameters for the feature selection algorithm

- First, default contant values are loaded by the config module.
- Then override the settings of select parameters as needed.

- The algorithm usually takes about 1+ minute per 1000 training iterations on CPU for the default 'sss' prior. The 'student' prior is faster.

In [ ]:
if run_or_load == "load":
    # loading the setup from the specified json
    constants, msg = load_constants_json(constants_filename)
    display(Markdown(msg))

else:
    # First load the default constants defined by the config module (including nonsensical values for your data)
    # And change the selected values below
    # This ensures that there is nothing missing
    constants = C.as_dict()
    constants["N_SAMPLES"] = X.shape[0]
    constants["N_FEATURES"] = X.shape[1]

    # ----------------------- SETUP OF THE FEATURE SELECTION PROCEDURE ------------------------------------------#

    # SPARSIITY = expected number of features per solution
    # only used to set up parameters PRIOR_SPARSITY and DESIRED_SPARSITY
    # to an identical value (can be overridden below if needed)
    constants["SPARSITY"] = 4

    # =======================================================
    #     Algorithm settings
    # =======================================================

    ### PRIOR_TYPE
    ### ...do not change unless you know what you are doing or nothing else works
    ### ...SSS = structured spike and slab priors
    ### ...method of 2nd recourse: 'student'
    constants["PRIOR_TYPE"] = "sss"

    ### PRIOR_SPARSITY = number of supporting dimensions for the 'sss' prior
    ### used only with the 'sss' prior
    constants["PRIOR_SPARSITY"] = constants[
        "SPARSITY"
    ]  # leave this setting unless there is a special reason to change it

    ### VAR_SPIKE and VAR_SLAB are only used with 'ss' and 'sss' prior
    ### ...VAR_SPIKE: parameter with the most influence on solution quality
    ### ...smaller VAR_SPIKE => more sparsity, i.e. fewer nonzero solutions
    ### ...increase VAR_SPIKE when all features converge to 0, typically in a uniform manner
    ### ...decrease VAR_SPIKE when there are too many nonzero features at the end of the run
    constants["VAR_SPIKE"] = 0.1
    ## Prior hyperparameters
    constants["VAR_SLAB"] = 100.0

    ### WEIGHT_SLAB and WEIGHT_SPIKE are only used with 'ss' prior
    constants["WEIGHT_SLAB"] = 0.9
    constants["WEIGHT_SPIKE"] = 0.1

    ### STUDENT_DF and STUDENT_SCALE are only used with the Student prior
    # Degrees of freedom (ν): controls tail heaviness and how “robust” the distribution is:
    #   low ν → very heavy tails, high ν → approaches Normal
    #   - Start moderately (e.g., 3–10). Very low (1–2) can slow convergence and produce very erratic large coefficients
    #   - Increase ν if solutions are too noisy or unstable
    #   - Decrease ν if you want to allow rare large effects
    # Scale (s): stretches or contracts the distribution
    #   acts like standard deviation when variance exists (ν > 2)
    #   - Decrease if too many features end up nonzero (tighten prior around zero).
    #   - Increase if all coefficients collapse near zero (prior overly restrictive).
    #   - Adjust s before ν if the issue is general spread rather than tail behavior.
    constants["STUDENT_DF"] = 5
    constants["STUDENT_SCALE"] = 1.0

    ## SGD optimization settings
    ### N_ITER = number of training iterations
    ### increase/decrease depending on both the ELBO and the convergence behavior of μs
    constants["N_ITER"] = 4000

    ## Regularization settings to make solutions more distinct
    ### IS_REGULARIZED = whether to use penalization based on average Jaccard similarity among solutions
    ### ...preferrably False or True with small LAMBDA_JACCARD at this moment
    constants["IS_REGULARIZED"] = True
    ### LAMBDA_JACCARD: larger values => more differentiated feature sets
    ### ...it is safer to use smaller values and increase only if needed
    ### ...if the ELBO converges to a too large value (in absolute terms), your lambda is probably too large
    constants["LAMBDA_JACCARD"] = 1000.0

    ## SGD optimization settings
    ## ...smaller learning rates require more iterations to converge but may yield better optimization processes
    ## ...batch size should be increased:
    #       - for larger datasets (to speed up convergence)
    #       - for datasets with a large percentage of missing values (to have enough complete data in each batch)
    #       - for datasets with small minory class sizes (to have enough samples of each class in every batch)
    constants["LEARNING_RATE"] = 0.002
    constants["BATCH_SIZE"] = int(np.min([16, constants["N_SAMPLES"]]))

    # ===============================================================
    #     Properties of the desired solutions and postprocessing
    # ===============================================================

    ### N_CANDIDATE_SOLUTIONS = number of candidate solutions
    ###                       = no. components of Gaussian mixture that approximate the posterior
    ### ...recommended to set as 2-3 times the expected number of solutions
    constants["N_CANDIDATE_SOLUTIONS"] = 8
    # Expected # of features per solution
    constants["DESIRED_SPARSITY"] = constants[
        "SPARSITY"
    ]  # if you want to change it, change the SPARSITY value above

    ### MIN_MU_THRESHOLD = minimum |μ| to consider a feature nonzero
    ### ...adjust based on the scale of your features and the convergence behavior
    constants["MIN_MU_THRESHOLD"] = 0.25

    # Settings for outlier detection
    # ...affect only outlier-based solution sets
    constants["USE_MEDIAN_FOR_OUTLIER_DETECTION"] = False  # recommended
    constants["OUTLIER_DEVIATION_THRESHOLDS"] = [2.0, 2.5, 3.0]

    # -------------------- NO NEED TO TOUCH CODE BELOW THIS CELL ---------------------------#

In [ ]:
# Run or loaded, show the configuration
display_current_config(
    constants=constants,
    constant_type="algorithm_and_postprocessing",
)
display_parameter_adjustment_summary()

# Run the feature selector on your data

There is no need to touch any code below this cell.

In [ ]:
if run_or_load == "run":
    # define the feature selector
    selector = BayesianFeatureSelector(
        n_features=constants["N_FEATURES"],
        n_components=constants["N_CANDIDATE_SOLUTIONS"],
        X=X,
        y=y,
        prior=constants["PRIOR_TYPE"],
        sss_sparsity=constants["PRIOR_SPARSITY"],
        var_slab=constants["VAR_SLAB"],
        var_spike=constants["VAR_SPIKE"],
        weight_slab=constants["WEIGHT_SLAB"],
        weight_spike=constants["WEIGHT_SPIKE"],
        student_df=constants["STUDENT_DF"],
        student_scale=constants["STUDENT_SCALE"],
        lr=constants["LEARNING_RATE"],
        batch_size=constants["BATCH_SIZE"],
        n_iter=constants["N_ITER"],
    )
    # run the selector
    history = selector.optimize(
        regularize=constants["IS_REGULARIZED"],
        lambda_jaccard=constants["LAMBDA_JACCARD"],
        verbose=verbose,
    )

    msg = save_selector_history_json(history, history_filename)
    display(Markdown(msg))
    msg = save_constants_json(constants, constants_filename)
    display(Markdown(msg))

else:
    history, msg = load_selector_history_json(history_filename)
    display(Markdown(msg))

In [ ]:
if show_search_history:
    show_algorithm_progress(
        history,
        original_feature_names_mapping=feature_to_name,
    )

    show_final_alphas(
        history,
        show_bar_plot=False,
        show_pie_chart=True,
    )

# Results

Analyze the feature selection process to extract solutions.

In [ ]:
full_solutions, top_solutions, outlier_solutions, final_parameters = recover_solutions(
    search_history=history,
    desired_sparsity=constants["DESIRED_SPARSITY"],
    min_mu_threshold=constants["MIN_MU_THRESHOLD"],
    original_feature_names_mapping=feature_to_name,
    use_median_for_outlier_detection=constants["USE_MEDIAN_FOR_OUTLIER_DETECTION"],
    outlier_deviation_thresholds=constants["OUTLIER_DEVIATION_THRESHOLDS"],
)

## Solutions by outlier detection

The outlier analysis helps identify features with unusually high importance values (mu, either positive or negative) in each component.

Ideally, the detected outliers are identical to the top solutions.

In [ ]:
outlier_features = {}
for deviation, df_outlier in outlier_solutions.items():
    show_solution_summary(
        solution_data=df_outlier,
        title=f"Outlier solutions for {deviation}",
        value_column="Feature",
    )

    show_unique_features_from_full_solutions(df_outlier)

    outlier_features[f"{deviation}"] = get_features_from_solutions(df_outlier)
    show_features_in_components(
        solutions=outlier_features[f"{deviation}"],
        features_to_show=get_unique_features(outlier_features[f"{deviation}"]),
    )

    if run_regressions_for_solutions:
        penalty = "l2"
        metrics = solve_any_regression(
            solutions=outlier_features[f"{deviation}"],
            df=df,
            response=response,
            apply_scaling=apply_scaling,
            penalty=penalty,
            verbose=False,
        )
        show_regression_metrics(
            metrics_df=metrics,
            title=f"Outlier features ({deviation}) - regression results on training data (penalty={penalty})",
            use_markdown=True,
        )

    display(Markdown("------------------------------------------------------------"))

## Full solutions

The 'full' solutions are the actual solutions (full sets of features) found by the algorithm. Their sparsity may not be as strong as desired.

In [ ]:
show_solution_summary(
    solution_data=full_solutions,
    title="Full solutions found by the feature selector, ordered by importance",
    value_column="Feature",
)

show_unique_features_from_full_solutions(full_solutions)

full_features = get_features_from_solutions(full_solutions)
show_features_in_components(
    solutions=full_features,
    features_to_show=get_unique_features(full_features),
)

if run_regressions_for_solutions:
    penalty = "l2"
    metrics = solve_any_regression(
        solutions=full_features,
        df=df,
        response=response,
        apply_scaling=apply_scaling,
        penalty=penalty,
        verbose=False,
    )
    show_regression_metrics(
        metrics_df=metrics,
        title=f"Full features - regression results on training data (penalty={penalty})",
        use_markdown=True,
    )

## Top solutions

The top solutions are just the most important features from the long solutions. The number of features selected is defined by the desired sparsity parameter.

In [ ]:
display(Markdown(f"### Required sparsity: {constants["DESIRED_SPARSITY"]}"))

show_solution_summary(
    solution_data=top_solutions,
    title=f"Top solutions with required sparsity {constants["DESIRED_SPARSITY"]}",
    value_column="Feature",
)

show_unique_features_from_full_solutions(top_solutions)

top_features = get_features_from_solutions(top_solutions)
show_features_in_components(
    solutions=top_features,
    features_to_show=get_unique_features(top_features),
)

if run_regressions_for_solutions:
    penalty = "l2"
    metrics = solve_any_regression(
        solutions=top_features,
        df=df,
        response=response,
        apply_scaling=apply_scaling,
        penalty=penalty,
        verbose=False,
    )
    show_regression_metrics(
        metrics_df=metrics,
        title=f"Top features - regression results on training data (penalty={penalty})",
        use_markdown=True,
    )

display(Markdown("------------------------------------------------------------"))

In [ ]:
# unify all feature lists for saving and displaying into a single dictionary
all_feature_dicts = [top_features, full_features] + list(outlier_features.values())
feature_dict_titles = [
    "Top features",
    "Full features",
] + [f"Outlier features ({deviation})" for deviation in outlier_features.keys()]

# Create a dictionary mapping feature list titles to their corresponding feature lists
# Structure: {title: {component_index: [features]}}
all_features_lists = {
    feature_dict_titles[i]: all_feature_dicts[i] for i in range(len(all_feature_dicts))
}

display(Markdown(f"## All available solutions"))
display(
    Markdown(
        f"**The following solution types are available:** {list(all_features_lists.keys())}"
    )
)

if verbose:
    for title, feature_dict in all_features_lists.items():
        display_feature_lists(
            feature_dict,
            title=title,
        )

In [ ]:
## Save all candidate solutions

if run_or_load == "run":
    msg = save_feature_lists_json(
        all_features_lists,
        features_filename_json,
    )
    display(Markdown(msg))

    # saving to txt for a well-readable file
    msg = save_feature_lists_txt(
        all_features_lists,
        features_filename_json.replace(".json", ".txt"),
    )
    display(Markdown(msg))

# More detailed analysis of the chosen solution

In [ ]:
display(Markdown(f"**Available solution types:** {list(all_features_lists.keys())}"))

In [ ]:
chosen_solution_type = "Outlier features (STD_2.0)"
if chosen_solution_type is not None:
    display(
        Markdown(
            f"## Regression for the chosen solution type **{chosen_solution_type}**"
        )
    )
    metrics = solve_any_regression(
        solutions=all_features_lists[chosen_solution_type],
        df=df,
        response=response,
        apply_scaling=apply_scaling,
        penalty=penalty,
        verbose=True,  # will show all details including the metrics summary
    )